In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "mb8gn4HBQfKyRV7nt+RG"),
    ca_certs=r"E:\Semantic_Search_Vector(BERT)_Embadding\elasticsearch-9.0.0\config\certs\http_ca.crt"

)
es.ping()

True

<h2><b>
Data Preprocessing</h2>


In [3]:
import pandas as pd
df= pd.read_csv("myntra_products_catalog.csv")
df.head(5)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [4]:
df.isna().value_counts()


ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           11597
                                                                                   True              894
Name: count, dtype: int64

In [5]:
df.fillna("None", inplace=True)


Convert the relevant field to Vector using BERT model




In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

e:\Semantic_Search_Vector(BERT)_Embadding\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df["DescriptionVector"] = model.encode(
    df["Description"].tolist(),
    batch_size=16,  # 16 for 2 GB GPU, 32 for 4 GB GPU
    show_progress_bar=True,
    
    normalize_embeddings=True  # good for dot product search
).tolist()


Batches: 100%|██████████| 781/781 [11:53<00:00,  1.09it/s] 


In [8]:
df.head(5)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.05290426313877106, -0.06897579878568649, -0..."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.01966804452240467, -0.05701494961977005, -..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.03742710128426552, -0.03298943117260933, -..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.0009177474421449006, -0.0936940386891365, ..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.030880963429808617, -0.053828418254852295,..."


In [9]:
len(df.loc[0, "DescriptionVector"])

768

In [10]:
df.columns

Index(['ProductID', 'ProductName', 'ProductBrand', 'Gender', 'Price (INR)',
       'NumImages', 'Description', 'PrimaryColor', 'DescriptionVector'],
      dtype='object')

In [11]:
len(df.columns)

9

In [12]:
es.ping()

True

### Create New Index in ElasticSearch!


In [13]:
record_list = df.to_dict("records")
record_list[0]

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black',
 'DescriptionVector': [0.05290426313877106,
  -0.06897579878568649,
  -0.03773300722241402,
  0.07500550895929337,
  0.012685127556324005,
  0.008912806399166584,
  0.045567456632852554,
  0.01856745034456253,
  -0.03904338553547859,
  -0.018211007118225098,
  -0.032046932727098465,
  0.07749397307634354,
  -0.029499894008040428,
  0.04555555433034897,
  0.041769582778

## data ingesting into the index 


In [14]:
es.indices.delete(index="all_products", ignore=[400, 404])


C:\Users\USER\AppData\Local\Temp\ipykernel_13576\4071565986.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="all_products", ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [15]:
from index_Mapping import indexMapping
indexMappingWithMappings = { "mappings": indexMapping }

es.indices.create(index="all_products", body=indexMappingWithMappings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

In [16]:
print(es.indices.get_mapping(index="all_products")["all_products"]["mappings"]["properties"]["Description"])
# Should output: {'type': 'text'}


{'type': 'text'}


In [17]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(f"Error indexing record {record['ProductID']}: {e}")


## Search the "Data"

In [25]:
input_keyword = "Mobile"
from numpy import array

query_vector = model.encode(input_keyword).tolist()

query = {
    "size":5,
    "_source": ["ProductName", "Description"],
    "query": {
        "knn": {
            "field": "DescriptionVector",  # Ensure this matches the field name in your index mapping
            "query_vector": query_vector,
            "k": 5,  # Number of nearest neighbors to retrieve
            "num_candidates": 500 # Optional: Adjust based on your requirements
        }
    }
}

res = es.search(index="all_products", body=query)



In [23]:
res['hits']['hits']

[{'_index': 'all_products',
  '_id': '10163543',
  '_score': 0.12544675,
  '_source': {'ProductName': 'BIANCA Pink Self Design Micro-Chenille Bath Rug',
   'Description': 'Set content: 1 bath rugColour: PinkPattern: Self designShape: Rectangular       GSM: 2100'}},
 {'_index': 'all_products',
  '_id': '10163563',
  '_score': 0.12507078,
  '_source': {'ProductName': 'BIANCA Turquoise Blue Self Design Micro-Chenille Bath Rug',
   'Description': 'Set content: 1 bath rugColour: Turquoise bluePattern: Self designShape: Rectangular       GSM: 2100'}},
 {'_index': 'all_products',
  '_id': '10195949',
  '_score': 0.122873284,
  '_source': {'ProductName': 'Chumbak Teal Green Cat Lady Printed iPhone 8 and SE Case',
   'Description': 'Teal green cat lady printed mobile caseSuitable for iPhone 8 and SE'}},
 {'_index': 'all_products',
  '_id': '10193831',
  '_score': 0.12276532,
  '_source': {'ProductName': 'Chumbak Unisex Teal Blue Cat Lady Printed iPhone X Case',
   'Description': 'Teal blue and 

In [24]:
es.ping()

True